In [77]:
!pip install tensorflow==2.14.0 #tensorflow-gpu
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: google-auth-oauthlib
    Found existing ins

In [78]:
!git clone https://github.com/ealeongomez/Time-Series-Forecasting-Using-the-Bayesian-Approach-and-Random-Fourier-Features.git

Cloning into 'Time-Series-Forecasting-Using-the-Bayesian-Approach-and-Random-Fourier-Features'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 117 (delta 0), reused 1 (delta 0), pack-reused 113 (from 1)
Receiving objects: 100% (117/117), 155.39 MiB | 12.03 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Updating files: 100% (33/33), done.


In [79]:
cd /content/Time-Series-Forecasting-Using-the-Bayesian-Approach-and-Random-Fourier-Features/Experiments-Wind/

/content/Time-Series-Forecasting-Using-the-Bayesian-Approach-and-Random-Fourier-Features/Experiments-Wind


In [80]:
ls -l

total 952
-rw-r--r-- 1 root root   1710 Dec  1 16:31 processing.py
-rw-r--r-- 1 root root   5782 Dec  1 16:31 RandomFourierFeatures.py
-rw-r--r-- 1 root root 459256 Dec  1 16:31 RFF-DataSet-1.ipynb
-rw-r--r-- 1 root root 499205 Dec  1 16:31 RFF-DataSet-2.ipynb


In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from tensorflow import keras


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from processing import create_features, split_data
from RandomFourierFeatures import Conv1dRFF

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score

In [87]:
input_dim = 1         # Input dimension
sequence_length = 30  # Sequence length
batch_size = 1        # Batch size
output_dim = 100      # Output dimension
kernel_size = 5       # Kernel size

#**Functions**

## PyTorch

In [88]:
def _get_random_features_initializer(initializer, shape, seed):
    def _get_cauchy_samples(loc, scale, shape):
        np.random.seed(seed)
        probs = np.random.uniform(low=0., high=1., size=shape)
        return loc + scale * np.tan(np.pi * (probs - 0.5))

    if isinstance(initializer, str):
        if initializer == "gaussian":
            return torch.nn.init.normal_(torch.empty(shape), mean=0.0, std=1.0)
        elif initializer == "laplacian":
            samples = _get_cauchy_samples(loc=0.0, scale=1.0, shape=shape)
            return torch.tensor(samples, dtype=torch.float32)
        else:
            raise ValueError(f'Unsupported kernel initializer {initializer}')


In [89]:
class Conv1dRFF_PT(nn.Module):
    def __init__(self, output_dim, kernel_size=3, scale=None, padding='VALID', normalization=True, function=True,
                 trainable_scale=False, trainable_W=False, seed=None, kernel='gaussian', **kwargs):
        super(Conv1dRFF_PT, self).__init__()
        self.output_dim = output_dim                   # Output dimension
        self.kernel_size = kernel_size                 # Convolutional operation size
        self.scale = scale                             # Kernel scale
        self.padding = padding                         # Padding type
        self.normalization = normalization             # Normalization flag
        self.function = function                       # Sine or cosine function
        self.trainable_scale = trainable_scale         # Scale trainability
        self.trainable_W = trainable_W                 # Kernel weights trainability
        self.seed = seed                               # Random seed
        self.initializer = kernel                      # Kernel initializer

        # Placeholder for weights to be initialized in `build`
        self.kernel = None
        self.bias = None
        self.kernel_scale = None

    def build(self, input_shape):
      input_dim = input_shape[1]

      kernel_initializer = _get_random_features_initializer(
          self.initializer,
          shape=(self.output_dim, input_dim, self.kernel_size),  # Cambiar el orden
          seed=self.seed
      )

      # Define kernel weights
      self.kernel = nn.Parameter(
          torch.tensor(kernel_initializer, dtype=torch.float32),
          requires_grad=self.trainable_W
      )

      # Define bias (tamaño debe coincidir con output_dim)
      self.bias = nn.Parameter(
          torch.empty(self.output_dim).uniform_(0.0, 2 * np.pi),
          requires_grad=self.trainable_W
      )

      # Set `scale` if not provided
      if not self.scale:
          if self.initializer == 'gaussian':
              self.scale = np.sqrt((input_dim * self.kernel_size ** 2) / 2.0)
          elif self.initializer == 'laplacian':
              self.scale = 1.0
          else:
              raise ValueError(f'Unsupported kernel initializer {self.initializer}')

      # Define scale as a trainable parameter
      self.kernel_scale = nn.Parameter(
          torch.tensor([self.scale], dtype=torch.float32),
          requires_grad=self.trainable_scale
      )

    def forward(self, inputs):
      scale = 1.0 / self.kernel_scale
      kernel = scale * self.kernel


      if self.padding == 'VALID':
          padding = 0
      elif self.padding == 'SAME':
          padding = self.kernel_size // 2
      else:
          raise ValueError(f'Unsupported padding type {self.padding}')

      outputs = F.conv1d(inputs, kernel, bias=self.bias, stride=1, padding=padding)

      if self.normalization:
          normalization_factor = np.sqrt(2 / self.output_dim)
          outputs = normalization_factor * (torch.cos(outputs) if self.function else torch.sin(outputs))
      else:
          outputs = torch.cos(outputs) if self.function else torch.sin(outputs)

      return outputs



In [90]:
inputs = torch.rand(batch_size, input_dim, sequence_length)
inputs.shape

torch.Size([1, 1, 30])

In [91]:
conv1d_rff = Conv1dRFF_PT(output_dim=output_dim,
                          kernel_size=kernel_size,
                          padding='VALID',  # Usaremos SAME para mantener la longitud
                          normalization=True,
                          function=True,
                          trainable_scale=True,
                          trainable_W=True,
                          seed=42,
                          kernel='gaussian'
)

conv1d_rff.build((batch_size, input_dim, sequence_length))

In [92]:
outputs = conv1d_rff(inputs)
outputs.shape

torch.Size([1, 100, 26])

## TensorFlow

In [97]:
inputs_np = np.array(inputs).reshape(batch_size, sequence_length, input_dim)
inputs_np.shape

(1, 30, 1)